# Plausability of Lottery Luck

#### [Dylan D. Daniels](http://statistics.berkeley.edu/people/dylan-david-daniels) and [Philip B. Stark](www.stat.berkeley.edu/~stark), Department of Statistics, University of California, Berkeley
#### Based on MATLAB code by [Skip Garibaldi](http://www.garibaldibros.com/)

This tool appraises whether it is plausible that a given individual won a set of lottery prizes honestly. 

This version uses data from a cell; other versions read data from .csv files.

The user inputs an upper bound on the number of players (for instance, one might assume that the number of people playing the lottery isn't greater than the number of residents of the state), and a tiny "threshold" probability.

The code outputs a lower bound on the amount all those people would have had to spend for any of them to have a tiny chance of winning so often, where "tiny" is the threshold number chosen by the user.

If the required spending amount is, for example, several times the median house price in the state, it may call into question whether the winner won honestly.

The current version can analyze data for only one gambler at a time. 

The code implements the mathematics described in the first link below. The third link is to a public lecture about the method, and results for reported lottery winners in Florida. 
The fourth and fifth links are news stories that relied on such calculations.

See:
+ Arratia, R., S. Garibaldi, L. Mower, and P.B. Stark, 2015. Some people have all the luck. _Mathematics Magazine_, _88_ 196–211. doi:10.4169/math.mag.88.3.196.c, Reprint: http://www.stat.berkeley.edu/~stark/Preprints/luck15.pdf http://www.jstor.org/stable/10.4169/math.mag.88.3.196
+ Arratia, R., S. Garibaldi, L. Mower, and P.B. Stark, 2015. Some people have all the luck &hellip; or do they? _MAA Focus_, August/September, 37–38. http://www.maa.org/sites/default/files/pdf/MAAFocus/Focus_AugustSeptember_2015.pdf
+ https://www.youtube.com/watch?v=s8cHHWNblA4
+  Lottery odds: To win, you’d have to be a loser. Lawrence Mower, _Palm Beach Post_, 28 March 2014. http://www.mypalmbeachpost.com/news/news/lottery-odds-to-win-youd-have-to-be-a-loser/nfL57
+ Against all Odds, Gavin Off and Adam Bell, _The Charlotte Observer_, 29 September 2016.
http://www.charlotteobserver.com/news/special-reports/against-all-odds/


## Instructions:
1. In the cell below, fill in data for the wins: p, n, and c

Each entry corresponds to one type of wager. "p" is the chance of winning that wager; "n" is the number of times the gambler collected on that wager; and "c" is the cost per ticket or play on that wager. The computations assume that the gambler did not win any dependent bets, for instance, two bets on the same drawing.

2. On the toolbar of this browser window (under the jupyter logo), click "Cell" --> "Run All". Wait a bit for your results to appear at the bottom of this page. 

In [1]:
p = [ 1./5760,\
    1./4800\
    ]
n = [ 370, 541+670+898+917+1121\
    ]
c = [ 20,\
    20\
    ]

In [2]:
from __future__ import print_function, division

# set the population size and overall cutoff probability
POPULATION = 6.794e6   # population of MA in 2015
THRESHOLD =  10**(-7) # one in ten million threshold
CUT = THRESHOLD / POPULATION # Bonferroni cutoff probability

In [3]:
import numpy as np
from scipy.stats import binom
from scipy.optimize import minimize

def constraintFn(p, n): # constraint function: probability of vector of wins must be at least CUT
    return lambda x: np.sum(binom.logsf(n-1, x, p)) - np.log(CUT)

def objectiveFn(c):  # construct function that gives cost of vector x of bets, for cost-per-bet vector c
    return lambda x: np.dot(x, c)

def solve(x0, upperBoundVec, p, n, c, eps, debugMode, maxiter, method='SLSQP'):  
    # invoke the constrained optimizer
    # 
    #    x0:     starting guess
    #    p:      vector of game probabilities
    #    n:      vector of number of wins of each game
    #    c:      vector of game costs
    #    eps:    stepsize for Hessian approximation
    #    debugMode: True for verbose output
    #    maxiter: maximum iterations in optimizer
    #    method: underlying minimization algorithm
    #       
    cons = ({'type': 'ineq', 'fun': constraintFn(p, n)})   # overall probability constraint
    bnds = tuple((n[i], upperBoundVec[i]) for i in range(len(n)))  # must bet at least n times to win n times
    return minimize(objectiveFn(c), x0, method=method, jac=(lambda x: c),
                    constraints=cons, bounds=bnds,
                    options={'disp': debugMode, 'maxiter': maxiter, 'eps': eps})

def solveProblem(tries=5, debugMode=False, epsilon = 1e-7, epsFac=8, maxiter=10**4):
    # Try up to epsFac values of the Hessian step size, related by powers of 10 (Hessian approximation step sizes)
    optimalValues = []     # candidate optima
    optimalProbs = []      # probabilities associated with those optima
    optimalSolutions = []  # detailed optimization output for candidate optima
    if debugMode:
        print("n: {} \np: {} \nc:".format(n,p,c))
    for meth in methods:   # try different optimization methods
        for epsIndex in range(epsFac):  # try different step sizes in the Hessian
            x0 = np.array(that/divisor) # starting guess
            for i in range(tries):
                while (np.sum(binom.logsf(n-1, x0, p)) - np.log(CUT)) < 0:  # ensure x0 is a feasible point
                    x0 = np.add(x0,np.ones_like(x0))  # increment every element of x
                if (debugMode):
                    print("method: {} try: {} \nx0: {} \nprobability {}:".format(meth,i,x0,\
                                np.exp(np.sum(binom.logsf(n-1, x0, p)))))
                optimOutput = solve(x0, that, p, n-1, c, epsilon*10**epsIndex, debugMode, maxiter, method=meth)
                if optimOutput['success']:
                    optimalValues.append(optimOutput['fun'])
                    attainedProb = np.exp(np.sum(binom.logsf(n-1, optimOutput['x'], p)))
                    optimalProbs.append(attainedProb)
                    optimalSolutions.append(optimOutput)
                    if debugMode:
                        print(optimOutput)
                        print("attained probability: {}".format(attainedProb))
                x0 = [np.random.randint(low=n[i], high=that[i]) for i in range(len(n))] # update x0 randomly
    if len(optimalValues) == 0:
        raise Exception('No candidate optimal solution found.')
    bestValue = np.min(optimalValues)
    largestProb = np.max(tuple(optimalProbs))
    if debugMode:
        print("\nFound {} candidate minima: {}".format(len(optimalValues), optimalValues))
        print("Best value: {}".format(bestValue))
    print("Everyone in the population of {} people would have to spend at least ${:,} to have probability {} that at least one would win these bets so often."
          .format(POPULATION, np.int(bestValue), THRESHOLD))
    return optimalValues, optimalProbs

In [ ]:
p = np.array(p)
n = np.array(n)
c = np.array(c)

that = n/p  # expected number of wagers on each bet required to win that bet n times

debugMode = True  # verbose output if True; set to False for less output

np.random.RandomState(seed=1234567890) # setting seed explicitly, for reproducibility

if debugMode:
    print ("initial t_hat: {} \ninitial log probability: {} \ntarget log probability {}".format(\
          that,\
          np.sum(binom.logsf(n-1, that, p)),\
          np.log(CUT)))

# 'that' will be used as an upper bound; ensure that it's compatible with the probability constraint
while np.sum(binom.logsf(n-1, that, p)) < np.log(CUT):
    that = 2*that
    if debugMode:
        print("estimated trials: {} log(tailprob): {}".format(that,np.sum(binom.logsf(n-1, that, p))))

divisor = 5 # initial value for optimizer is expected number divided by divisor (modified to ensure feasibility)
methods = ['SLSQP','COBYLA']  # COBYLA will ignore the individual bounds, but should honor the probability constraint

if debugMode:
    print ("adjusted t_hat: {} \nadjusted probability: {}".format(that,np.exp(np.sum(binom.logsf(n-1, that, p)))))

initial t_hat: [  2131200.  19905600.] 
initial log probability: -1.36843585088 
target log probability -31.8496460787
adjusted t_hat: [  2131200.  19905600.] 
adjusted probability: 0.254504731737


In [ ]:
optimalValues, optimalProbs = solveProblem(tries = 5, debugMode=debugMode, epsilon = 1e-7, epsFac=8, maxiter=10**4)

n: [ 370 4147] 
p: [ 0.00017361  0.00020833] 
c:


//anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:815: RuntimeWarning: divide by zero encountered in log
  return log(self._sf(x, *args))


In [ ]:
# version information
%load_ext version_information
%version_information scipy, numpy, pandas, matplotlib, notebook